In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import wordpunct_tokenize
nltk.download('stopwords')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/summerlong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score



In [24]:
tweets = pd.read_csv('data/obamacare_tw_19_23.csv')
yelp = pd.read_csv('data/yelp_true_sample_100k.csv')

In [25]:
import nltk
from nltk.corpus import stopwords
import re

def preprocess_text(text):
    # Remove links
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove '\r' and '\n'
    text = text.replace('\r', '').replace('\n', '')
    
    # Tokenize text, ensuring it is not case insensitive
    tokens = nltk.word_tokenize(text.lower())
    
    # Remove stop words, words starting with "@"
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Join filtered tokens back into a string
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text


# apply function to 'text' column in yelp_data
tweets['Text_Processed'] = tweets['Text'].apply(preprocess_text)

In [26]:
import joblib
tfidf_vectorizer = joblib.load('models/tfidf_vectorizer.pkl')

In [27]:
xgboost = joblib.load('models/xgboostv1.pkl')


In [28]:
tweets_sampled_index = pd.read_csv('data/tweetmanualsample.csv')

In [29]:
tweets_sampled_index

,index,sentiment
0,16528,0
1,19792,0
2,93112,1
3,79706,0
4,91551,0
...,...,...
195,51295,0
196,47360,0
197,94472,1
198,81154,1


In [30]:
index_list = tweets_sampled_index['index'].tolist()

In [31]:
tweets_sample = tweets.loc[index_list]

In [32]:
tweets_sample = tweets_sample.reset_index()

In [33]:
merged_df = tweets_sample.merge(tweets_sampled_index, left_on='index', right_on='index')


In [34]:
merged_df

,index,Id,Date,Text,Text_Processed,sentiment
0,16528,1140752233241415682,2019-06-17,@realDonaldTrump So when are you and Republica...,@ realdonaldtrump republicans going `` repeal ...,0
1,19792,1150786889118617601,2019-07-15,@BernieSanders FINALLY! YOU ADMIT YOU ARE AT F...,@ berniesanders finally ! admit fault obamacar...,0
2,93112,1571969173323931658,2022-09-19,@DeviousKindly @PoliticsCourage @AlonaGutman @...,@ deviouskindly @ politicscourage @ alonagutma...,1
3,79706,1422150880858501124,2021-08-02,It’s just like explaining “socialist Obamacare...,’ like explaining “ socialist obamacare ” affo...,0
4,91551,1554176905892532234,2022-08-01,@Solja42 @Andie00471 @desmondalan I voted blue...,@ solja42 @ andie00471 @ desmondalan voted blu...,0
...,...,...,...,...,...,...
195,51295,1290104622133125120,2020-08-03,And also he took away ObamaCare right before p...,also took away obamacare right pandemic actual...,0
196,47360,1272391779543846914,2020-06-15,@LGoralska @ErrolWebber @realDonaldTrump @POTU...,@ lgoralska @ errolwebber @ realdonaldtrump @ ...,0
197,94472,1582065199086850048,2022-10-17,President Biden Announces Fix to Family Glitch...,president biden announces fix family glitch ob...,1
198,81154,1445084919726698506,2021-10-04,The South Florida Sun Sentinel reports former ...,south florida sun sentinel reports former pres...,1


In [35]:
tweets_sample_tfidf = tfidf_vectorizer.transform(merged_df['Text_Processed'])

In [36]:
# Extract true labels from the 'sentiment' column
true_labels = merged_df['sentiment']

# Make predictions
predictions = xgboost.predict(tweets_sample_tfidf)

# Generate classification report
report = classification_report(true_labels, predictions)

print(report)

              precision    recall  f1-score   support

           0       0.83      0.36      0.50       135
           1       0.39      0.85      0.53        65

    accuracy                           0.52       200
   macro avg       0.61      0.60      0.51       200
weighted avg       0.68      0.52      0.51       200



In [37]:
auc_roc_score = roc_auc_score(true_labels, predictions)

print(f"AUC-ROC score: {auc_roc_score}")

AUC-ROC score: 0.6008547008547009


In [38]:
merged_df['xgb_pred'] = predictions

In [39]:
merged_df

,index,Id,Date,Text,Text_Processed,sentiment,xgb_pred
0,16528,1140752233241415682,2019-06-17,@realDonaldTrump So when are you and Republica...,@ realdonaldtrump republicans going `` repeal ...,0,0
1,19792,1150786889118617601,2019-07-15,@BernieSanders FINALLY! YOU ADMIT YOU ARE AT F...,@ berniesanders finally ! admit fault obamacar...,0,1
2,93112,1571969173323931658,2022-09-19,@DeviousKindly @PoliticsCourage @AlonaGutman @...,@ deviouskindly @ politicscourage @ alonagutma...,1,0
3,79706,1422150880858501124,2021-08-02,It’s just like explaining “socialist Obamacare...,’ like explaining “ socialist obamacare ” affo...,0,0
4,91551,1554176905892532234,2022-08-01,@Solja42 @Andie00471 @desmondalan I voted blue...,@ solja42 @ andie00471 @ desmondalan voted blu...,0,1
...,...,...,...,...,...,...,...
195,51295,1290104622133125120,2020-08-03,And also he took away ObamaCare right before p...,also took away obamacare right pandemic actual...,0,1
196,47360,1272391779543846914,2020-06-15,@LGoralska @ErrolWebber @realDonaldTrump @POTU...,@ lgoralska @ errolwebber @ realdonaldtrump @ ...,0,1
197,94472,1582065199086850048,2022-10-17,President Biden Announces Fix to Family Glitch...,president biden announces fix family glitch ob...,1,1
198,81154,1445084919726698506,2021-10-04,The South Florida Sun Sentinel reports former ...,south florida sun sentinel reports former pres...,1,1


In [40]:
randomforest = joblib.load('models/randomforestv2.pkl')


In [41]:
predictions2 = randomforest.predict(tweets_sample_tfidf)

report2 = classification_report(true_labels, predictions2)

print(report2)

              precision    recall  f1-score   support

           0       1.00      0.01      0.01       135
           1       0.33      1.00      0.49        65

    accuracy                           0.33       200
   macro avg       0.66      0.50      0.25       200
weighted avg       0.78      0.33      0.17       200



In [42]:
auc_roc_score2 = roc_auc_score(true_labels, predictions2)

print(f"AUC-ROC score: {auc_roc_score2}")

AUC-ROC score: 0.5037037037037038


In [43]:
merged_df['rf_pred'] = predictions2

In [72]:
logisticregression = joblib.load('models/best_logistic_clf.pkl')

predictions3 = logisticregression.predict(tweets_sample_tfidf)

report3 = classification_report(true_labels, predictions3)

print(report3)

/usr/local/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


ValueError: X has 65910 features, but LogisticRegression is expecting 66095 features as input.

In [44]:
svm = joblib.load('models/best_svm.pkl')

predictions4 = svm.predict(tweets_sample_tfidf)

report4 = classification_report(true_labels, predictions4)

print(report4)

              precision    recall  f1-score   support

           0       0.82      0.43      0.56       135
           1       0.40      0.80      0.54        65

    accuracy                           0.55       200
   macro avg       0.61      0.61      0.55       200
weighted avg       0.68      0.55      0.55       200



In [45]:
auc_roc_score4 = roc_auc_score(true_labels, predictions4)

print(f"AUC-ROC score: {auc_roc_score4}")


AUC-ROC score: 0.6148148148148148


In [46]:
merged_df['svm_pred'] = predictions4

In [84]:
neural_net = joblib.load('models/neural_net.pkl')

predictions5 = neural_net.predict(tweets_sample_tfidf)

report5 = classification_report(true_labels, predictions5)

print(report5)

TypeError: __randomstate_ctor() takes from 0 to 1 positional arguments but 2 were given

In [48]:
merged_df.to_csv('tweets_with_predictions.csv', index=False)